
# Recursivity cascades

## Objectives
- Describe how recursivity propagates structural echoes across nested EPIs.
- Capture the deterministic ΔNFR hook required by Phase-2 remesh cascades.
- Provide reproducible traces that verify multi-scale coherence after each recursion step.

## Phase-2 dependencies
- [Phase-2 integration notes](../fase2_integration.md) — the recursive remesh orchestration consumes the same trace structure emitted here.
- :mod:`tnfr.structural` — Phase-2 triggers the same canonical recursivity→reception→coherence→resonance→transition pipeline when stitching sub-EPIs.
- :mod:`tnfr.operators.definitions` — glyph assignments stay identical, ensuring the deterministic trace matches runtime expectations.

## Theoretical exposition
Recursivity replays structural motifs across scales. Each application copies the local ΔNFR ledger into progressively finer sub-EPIs, adjusting νf so that coherence remains invariant. By scripting the ΔNFR injections per level (root → child → leaf) we can verify that the resulting EPI and νf trajectories evolve proportionally and that the cascade trace records the nested ratios required for Phase-2 remeshing.

## Deterministic smoke check
The notebook builds a three-level chain, executes the canonical recursivity→reception→coherence→resonance→transition segment, and records EPI/νf pairs after each application. The before/after snapshots and the trace grouped by level are deterministic, allowing CI to detect any modification to the cascade grammar or ΔNFR propagation rules.


In [ ]:

from collections import defaultdict
from typing import Dict, Iterable, List, Tuple

from tnfr.constants import DNFR_PRIMARY, EPI_PRIMARY, VF_PRIMARY
from tnfr.dynamics import set_delta_nfr_hook
from tnfr.structural import Coherence, Reception, Recursivity, Resonance, Transition, create_nfr, run_sequence

G, root = create_nfr("cascade-root", epi=0.5, vf=1.02, theta=0.0)
_, child = create_nfr("cascade-child", graph=G, epi=0.34, vf=1.08, theta=0.12)
_, leaf = create_nfr("cascade-leaf", graph=G, epi=0.26, vf=1.12, theta=-0.18)
G.add_edge(root, child)
G.add_edge(child, leaf)

scripts: Dict[str, Iterable[Tuple[float, float]]] = {
    root: iter([(0.04, 0.02), (0.03, 0.015), (0.025, 0.012), (0.02, 0.01), (0.015, 0.008)]),
    child: iter([(0.032, 0.015), (0.024, 0.01), (0.02, 0.008), (0.016, 0.006), (0.012, 0.005)]),
    leaf: iter([(0.025, 0.01), (0.018, 0.008), (0.015, 0.006), (0.012, 0.005), (0.01, 0.004)]),
}

cascade_trace: List[Tuple[str, float, float]] = []
labels = {root: "root", child: "child", leaf: "leaf"}


def snapshot() -> dict[str, dict[str, float]]:
    return {
        labels[node]: {
            "EPI": round(float(G.nodes[node][EPI_PRIMARY]), 6),
            "νf": round(float(G.nodes[node][VF_PRIMARY]), 6),
        }
        for node in (root, child, leaf)
    }


def cascade_hook(graph):
    for node_id, iterator in scripts.items():
        dnfr, vf_shift = next(iterator, (0.0, 0.0))
        data = graph.nodes[node_id]
        vf_before = float(data[VF_PRIMARY])
        epi_before = float(data[EPI_PRIMARY])
        data[DNFR_PRIMARY] = dnfr
        data[EPI_PRIMARY] = epi_before + vf_before * dnfr
        data[VF_PRIMARY] = vf_before + vf_shift
        cascade_trace.append(
            (labels[node_id], round(float(data[EPI_PRIMARY]), 6), round(float(data[VF_PRIMARY]), 6))
        )


before = snapshot()
set_delta_nfr_hook(G, cascade_hook, note="recursivity cascade smoke")
run_sequence(G, root, [Recursivity(), Reception(), Coherence(), Resonance(), Transition()])
after = snapshot()

trace_by_level: Dict[str, List[Tuple[float, float]]] = defaultdict(list)
for label, epi_val, vf_val in cascade_trace:
    trace_by_level[label].append((epi_val, vf_val))

{
    "before": before,
    "after": after,
    "trace": {key: trace_by_level[key] for key in ("root", "child", "leaf")},
}
